In [ ]:
import numpy as np
import pandas as pd
import bokeh
from bokeh.models import FactorRange, ColumnDataSource, LinearColorMapper,  ColorBar
from bokeh.transform import transform

from tqdm.notebook import tqdm

import scipy.stats as st
import iqplot

import Bio.Seq

bokeh.io.output_notebook()

The purpose of this code is to investigate the diversity of deep sequenced substitution libraries. The nucleotide counts are generated using the `FASTQ_seq_counting.ipynb` notebook. 

We can write a function to extract amino acid information from the nucleotide. 

In [ ]:
def data_ops(dataframe, site, stage):
    AA_seq = np.empty_like(dataframe['barcode'])

    for i, barcode in tqdm(enumerate(dataframe['barcode'])):
        AA_seq[i] = Bio.Seq.translate(dataframe['barcode'][i])

    dataframe['aa'] = AA_seq.astype('str')
    
    dataframe['site'] = site
    
    dataframe['stage'] = stage
    
    columns = ['site', 'stage', 'barcode', 'aa', 'counts']
    
    dataframe = dataframe[columns]
    
    return(dataframe)

We can set some parameters about AAV9 and amino acids for later reference. 

In [ ]:
#Initialize the AAV9 AA sequence between each substitution for comparison
aav9_452 = 'NGSGQNQ'
aav9_492 = 'TVTQNNN'
aav9_585 = 'QSAQAQA'

#Create a dictionary and a list containing amino acids
AA_dict = {'A':0,
           'C':1,
           'D':2,
           'E':3,
           'F':4, 
           'G':5,
           'H':6,
           'I':7,
           'K':8,
           'L':9,
           'M':10,
           'N':11,
           'P':12,
           'Q':13,
           'R':14,
           'S':15,
           'T':16,
           'V':17,
           'W':18,
           'Y':19,
           '*':20}

aa_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','*']

We want to analyze each of the files generated from the counting notebook and include metadata in the process. 

In [ ]:
# df_452_ll = pd.read_csv('analysis/452ll_pairedend1.csv')
# df_452_ll = data_ops(df_452_ll, '452', 'll')

# df_492_ll = pd.read_csv('analysis/492ll_pairedend1.csv')
# df_492_ll = data_ops(df_492_ll, '492', 'll')

# df_585_ll = pd.read_csv('analysis/585ll_pairedend1.csv')
# df_585_ll = data_ops(df_585_ll, '585', 'll')

# df_452_vg = pd.read_csv('analysis/452vg_pairedend1.csv')
# df_452_vg = data_ops(df_452_vg, '452', 'vg')

# df_492_vg = pd.read_csv('analysis/492vg_pairedend1.csv')
# df_492_vg = data_ops(df_492_vg, '492', 'vg')

# df_585_vg = pd.read_csv('analysis/585vg_pairedend1.csv')
# df_585_vg = data_ops(df_585_vg, '585', 'vg')

# df = pd.concat([df_452_ll, df_492_ll, df_585_ll, df_452_vg, df_492_vg, df_585_vg], ignore_index=True)

Due to the time investment, we will save the output so that this doesn't need to be re-run. 

In [ ]:
#df.to_csv('analysis/diversity.csv', index=False)

In future runs, we can load the csv for analysis.

In [ ]:
df = pd.read_csv('analysis/diversity.csv')

First, we want to examine the cumulative density function of the read counts to examine the diversity across libraries. We examine the linear libraries first.

In [ ]:
inds = (df['stage'] == 'll')

df_plot = df.loc[inds].sample(500000).sort_values('site')

p = iqplot.ecdf(data = df_plot, q = 'counts', cats = 'site', x_axis_type="log", style = 'staircase', x_axis_label='linear library counts', palette = ['#5ec962','#21918c','#440154'])

bokeh.io.show(p)

Next, we examine the read counts of the viral genomes.

In [ ]:
inds = df['stage'] == 'vg'

df_plot = df.loc[inds].sample(500000).sort_values('site')

p = iqplot.ecdf(data = df_plot, q = 'counts', cats = 'site', x_axis_type="log", style = 'staircase', x_axis_label='viral genome counts', palette = ['#5ec962','#21918c','#440154'])

bokeh.io.show(p)

Next, we can examine the hamming distance for the libraries to see how much each site varies from AAV9. 

First, we need to set which site and stage we want to examine. We index the dataframe at these locations. We'll create a dataframe that contains all the sites for linear library. The viral genome hamming distance can be examined by changing stage to 'vg'.

In [ ]:
sites = [452, 492, 585]
stage = 'vg'

We can then calculate the hamming distance across each substitution.

In [ ]:
df_plot = pd.DataFrame()

for site in sites: 
    inds = (df['site'] == site) & (df['stage'] == stage)

    lst = set(df.loc[inds, 'aa'].values)

    hamming_dist = {k: 0 for k in lst}

    for sequence in tqdm(lst):

        for i in range(7):
            if sequence[i] != globals()['aav9_' + str(site)][i]:
                hamming_dist[sequence] += 1

    df_hd = pd.DataFrame(hamming_dist.items(), columns = ['sequence', 'hamming_dist'])
    
    df_hd['site'] = site
    df_hd['stage'] = stage
    
    df_plot = pd.concat([df_plot, df_hd])

We can plot the resulting data.

In [ ]:
p = iqplot.ecdf(df_plot.sample(500000).sort_values('site'), q = 'hamming_dist', cats = 'site', style = 'staircase', palette = ['#5ec962','#21918c','#440154'], x_axis_label='linear library hamming distance')

p.output_backend = 'svg'

bokeh.io.show(p)

Next, we want to examine the amino acid residue frequency by position. To do this, we must first set an index of the specific site and stage (linear library or viral genomes) we want to examine.

In [ ]:
site = 452

stage = 'll'

First, we can determine how many times the amino acids occured in the library, as a total.

In [ ]:
inds = (df['site'] == site) & (df['stage'] == stage)

total_aa_matrix = np.zeros((21,7), dtype=int)

for aa in df.loc[inds,'aa']:
    for i in range(7):
        total_aa_matrix[AA_dict[aa[i]],i] += 1

Next, we can divide by the sum of each column to get the frequency.

In [ ]:
aa_freq_matrix = np.zeros_like(total_aa_matrix).astype(float)

for j in range(7):
    aa_freq_matrix[:,j] = total_aa_matrix[:,j]/sum(total_aa_matrix[:,j])

Then, we can stack the dataframe to for convenience during plotting.

In [ ]:
df_hm = pd.DataFrame(aa_freq_matrix, 
                     columns=np.arange(site, site + 7, 1).astype(str), 
                     index=aa_list)

df_hm.index.name = 'Amino Acid'
df_hm.columns.name = 'Position'

df_hm = df_hm.stack().rename("value").reset_index()

Then, we can generate the heatmap plot from this data.

In [ ]:
mapper = LinearColorMapper(
    palette='Magma256', low=df_hm['value'].min(), high=df_hm['value'].max())

# Define a figureabsabs
p = bokeh.plotting.figure(
    plot_width=500,
    plot_height=500,
    x_range=list(df_hm["Position"].drop_duplicates()),
    y_range=list(reversed(list(df_hm["Amino Acid"].drop_duplicates()))),
    toolbar_location='right',
    tools = "pan,wheel_zoom,box_zoom,reset,save",
    x_axis_location="above")

# Create rectangle for heatmap
p.rect(
    x="Position",
    y="Amino Acid",
    width=1,
    height=1,
    source=ColumnDataSource(df_hm),
    line_color=None,
    fill_color=transform('value', mapper))

# Add legend
color_bar = ColorBar(
    color_mapper=mapper,
    location=(0, 0))

p.add_layout(color_bar, 'right')

p.output_backend = "svg"

bokeh.io.show(p)

The above cells can be re-run for each library and stage.